# Consolidation

In [1]:
!pip install -U transformers==4.48.3 -qqq

### Dependencias

In [2]:
import os

if os.environ.get("TRANSFORMERS_CACHE"):
    os.environ["HF_HOME"] = os.environ.pop("TRANSFORMERS_CACHE")

import math
import re
import shutil
from dataclasses import dataclass
from itertools import product
from typing import Any, Dict, List, Optional

import flash_attn
import pandas as pd
import tabulate
import torch
import torch.nn as nn
from datasets import Dataset, load_dataset
from flash_attn import flash_attn_qkvpacked_func
from huggingface_hub import Repository, whoami
from pydantic import BaseModel, field_validator
from torch.optim import AdamW
from tqdm.auto import tqdm
from transformers import (
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    get_linear_schedule_with_warmup,
)

### Parâmetros

In [3]:
AMOUNT_OF_NEWS = 3535
AMOUNT_OF_SENTENCES = 3456

MODEL_ID = "answerdotai/ModernBERT-base"  # "neuralmind/bert-base-portuguese-cased"
DATASET_ID = "emdemor/news-of-the-brazilian-newspaper"
USERNAME = "emdemor"
TOKENIZER_PATH = "domain_tokenizer"
TESTING = True
FLASH_ATTENTION = False
PUSH_INTERVAL = 10_000 if TESTING else 100_000
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

MODEL_NAME = MODEL_ID.split("/")[-1]
TRAINED_MODEL_PATH = f"{MODEL_NAME}-ptbr-{'test' if TESTING else 'full'}"

# A probabilidade de MLM determina quantos tokens serão mascarados durante o treinamento
# Usar diferentes probabilidades pode ajudar o modelo a aprender melhor
# Aqui podemos aplicar o conceito de "Aprendizagem Curricular", onde:
#   1. Começamos com tarefas mais simples
#   2. Gradualmente aumentamos a dificuldade
#   3. Permitimos que o modelo construa competências de forma incremental
#
# Como prever 14% dos tokens mascarados de uma sentença é mais fácil do que prever 30%,
# podemos começar do menor para o maior
#
# Benefícios desta abordagem:
#  1. O modelo primeiro aprende padrões básicos com menos tokens mascarados
#  2. Gradualmente enfrenta desafios mais complexos à medida que ganha competência
#  3. Potencialmente leva a um aprendizado mais estável e eficaz
MLM_PROBABILITIES = [0.05, 0.10, 0.15, 0.20, 0.30]

## Helpers and Functions

In [4]:
def split_into_sentences(text: str) -> List[str]:
    """Split text into sentences."""
    return [
        sentence.strip()
        for sentence in re.split(r"(?<=[.!?])\s+", text)
        if sentence.strip()
    ]

In [5]:
def set_attention(model):
    def check_flash_attention_support():
        if not torch.cuda.is_available():
            return False
        try:
            qkv = torch.randn(1, 1, 3, 16, 64, dtype=torch.float16, device="cuda")
            flash_attn_qkvpacked_func(qkv, causal=False)
            return True
        except RuntimeError as e:
            print("Flash Attention não é compatível:", str(e))
            return False

    if FLASH_ATTENTION and check_flash_attention_support():
        print("Replacing standard attention with FlashAttention...")
        for module in model.modules():
            if isinstance(module, nn.MultiheadAttention):
                module.attention = FlashAttention()
        print("FlashAttention integrated.")

    return model

In [6]:
def check_vocab_size(tokenizer, model):
    max_token_id = max(tokenizer.get_vocab().values())
    print("Maior ID no tokenizador:", max_token_id)
    print("Tamanho do vocabulário do modelo:", model.config.vocab_size)
    assert max_token_id < model.config.vocab_size, "IDs de tokens fora do intervalo!"

In [7]:
class TrainingConfig(BaseModel):
    dataset_size: int
    num_train_epochs: int
    num_chunks: int
    train_batch_size_per_device: int
    gradient_accumulation_steps: int
    eval_size_ratio: float
    total_save_limit: int

    @field_validator("num_chunks")
    def validate_num_chunks(cls, v, info):
        data = info.data
        if (
            "dataset_size" in data
            and "dataset_size" in data
            and "eval_size_ratio" in data
        ):
            dataset_size = data["dataset_size"]
            eval_size_per_chunk = int(data["dataset_size"] * data["eval_size_ratio"])
            available_size = dataset_size - eval_size_per_chunk * v
            if available_size < v:
                raise ValueError(
                    f"available_size ({available_size}) deve ser maior ou igual a num_chunks ({v})"
                )
        return v

    @property
    def effective_batch_size(self):
        return self.train_batch_size_per_device * self.gradient_accumulation_steps

    @property
    def total_steps_per_epoch(self):
        return math.ceil(self.dataset_size / self.effective_batch_size)

    @property
    def total_train_steps(self):
        return self.total_steps_per_epoch * self.num_train_epochs

    @property
    def eval_size_per_chunk(self):
        """Tamanho do dataset de avaliação em cada chunk"""
        return int(self.dataset_size * self.eval_size_ratio / self.num_chunks)

    @property
    def available_size(self):
        return self.dataset_size - self.eval_size_per_chunk * self.num_chunks

    @property
    def eval_size(self):
        return self.dataset_size - self.available_size

    @property
    def chunk_size(self):
        return self.dataset_size // self.num_chunks

    @property
    def chunk_train_size(self):
        return self.available_size // self.num_chunks

    def __repr(self):
        data = [
            ["num_train_epochs", self.num_train_epochs],
            ["dataset_size", self.dataset_size],
            ["num_chunks", self.num_chunks],
            ["chunk_size", self.chunk_size],
            ["chunk_train_size", self.chunk_train_size],
            ["eval_size_per_chunk", self.eval_size_per_chunk],
            ["eval_size_ratio", self.eval_size_ratio],
            ["available_size", self.available_size],
            ["eval_size", self.eval_size],
            ["train_batch_size_per_device", self.train_batch_size_per_device],
            ["gradient_accumulation_steps", self.gradient_accumulation_steps],
            ["total_save_limit", self.total_save_limit],
            ["effective_batch_size", self.effective_batch_size],
            ["total_steps_per_epoch", self.total_steps_per_epoch],
            ["total_train_steps", self.total_train_steps],
        ]

        return tabulate.tabulate(data, headers=["Attribute", "Value"], tablefmt="grid")

    def __repr__(self):
        return self.__repr()

    def __str__(self):
        return self.__repr()

### Preparar a Base de Dados

In [8]:
"""Load and prepare the dataset for training."""

raw_dataset = load_dataset(DATASET_ID, split="train")
df = raw_dataset.to_pandas().sample(frac=1).reset_index(drop=True)
sample_df = df.sample(min(AMOUNT_OF_NEWS, len(df)))
combined_texts = sample_df["text"].to_list() + sample_df["title"].to_list()
sentences = [
    phrase for text in combined_texts if text for phrase in split_into_sentences(text)
]
sentences_sample = pd.Series(sentences).sample(AMOUNT_OF_SENTENCES).to_list()
dataset = Dataset.from_dict({"text": sentences_sample})

### Setup model and tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    TOKENIZER_PATH, clean_up_tokenization_spaces=False
)
config = AutoConfig.from_pretrained(MODEL_ID)
config.torch_dtype = torch.float16
model = AutoModelForMaskedLM.from_pretrained(MODEL_ID, config=config)
model.resize_token_embeddings(len(tokenizer))
model.to(DEVICE)
model = set_attention(model)
check_vocab_size(tokenizer, model)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Maior ID no tokenizador: 32767
Tamanho do vocabulário do modelo: 32768


In [10]:
training_config = TrainingConfig(
    num_train_epochs=3,
    dataset_size=len(dataset),
    num_chunks=len(MLM_PROBABILITIES),
    train_batch_size_per_device=4,
    gradient_accumulation_steps=2,
    eval_size_ratio=0.10,
    total_save_limit=2,
    estimated_dataset_size_in_rows=len(dataset),
)


print(training_config)

+-----------------------------+---------+
| Attribute                   |   Value |
+=============================+=========+
| num_train_epochs            |     3   |
+-----------------------------+---------+
| dataset_size                |  3456   |
+-----------------------------+---------+
| num_chunks                  |     5   |
+-----------------------------+---------+
| chunk_size                  |   691   |
+-----------------------------+---------+
| chunk_train_size            |   622   |
+-----------------------------+---------+
| eval_size_per_chunk         |    69   |
+-----------------------------+---------+
| eval_size_ratio             |     0.1 |
+-----------------------------+---------+
| available_size              |  3111   |
+-----------------------------+---------+
| eval_size                   |   345   |
+-----------------------------+---------+
| train_batch_size_per_device |     4   |
+-----------------------------+---------+
| gradient_accumulation_steps |   

### Tokenize dataset

In [11]:
def tokenize_function(examples, target_column="text"):
    return tokenizer(
        examples[target_column],
        # No truncation and max_length to allow dynamic padding truncation=True, max_length=chunk_size, padding="longest",
        return_special_tokens_mask=True,
    )


def tokenize_dataset(dataset):

    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset.column_names,
    )

    return tokenized_dataset


tokenized_dataset = tokenize_dataset(dataset)

Map:   0%|          | 0/3456 [00:00<?, ? examples/s]

### Training

## Definindo o DataCollator

### O que é um DataCollator no Contexto de Processamento de Linguagem Natural

Um DataCollator é uma componente fundamental no pipeline de treinamento de modelos de linguagem natural, especialmente quando trabalhamos com bibliotecas como a Hugging Face Transformers. É uma função ou classe responsável por agrupar vários exemplos individuais em um único lote (batch) que pode ser processado eficientemente pelo modelo durante o treinamento. O nome "collator" vem do verbo "collate", que significa reunir e organizar informações.

#### Por que Precisamos de DataCollators?

Quando treinamos modelos de linguagem, geralmente trabalhamos com textos de comprimentos variados. No entanto, os modelos de deep learning, especialmente aqueles implementados em PyTorch ou TensorFlow, esperam tensores de forma consistente (retangular). Isso cria um desafio:

1. Como transformar exemplos de tamanhos diferentes em uma estrutura uniforme?
2. Como fazer isso de maneira eficiente sem desperdiçar recursos computacionais?

É aqui que entra o DataCollator.

#### Funções Principais de um DataCollator

##### 1. Padding (Preenchimento)

A função mais importante é o preenchimento (padding). Como as sequências têm comprimentos diferentes, o DataCollator adiciona tokens especiais (geralmente 0s) para que todas as sequências em um lote tenham o mesmo comprimento.

Por exemplo, se tivermos estas três sequências:
```
[101, 2054, 2003, 102]  # "What is"
[101, 2054, 2003, 2023, 2003, 102]  # "What is this"
[101, 2054, 2003, 2023, 102]  # "What is this"
```

O DataCollator as transformaria em:
```
[101, 2054, 2003, 102, 0, 0]
[101, 2054, 2003, 2023, 2003, 102]
[101, 2054, 2003, 2023, 102, 0]
```

##### 2. Mascaramento para MLM (no caso de DataCollatorForLanguageModeling)

No código que você compartilhou, está sendo usado o `DataCollatorForLanguageModeling`, que é especializado para treinar modelos de linguagem mascarada (como BERT). Além do padding, ele:

- Seleciona aleatoriamente uma porcentagem dos tokens (definida por `mlm_probability`)
- Substitui esses tokens por um token especial [MASK]
- Mantém o registro dos tokens originais para calcular a perda durante o treinamento

Por exemplo, com `mlm_probability=0.15`:
```
Original: [101, 2054, 2003, 2023, 2003, 102]
Mascarado: [101, 2054, [MASK], 2023, 2003, 102]  # "2003" foi mascarado
```

##### 3. Otimização de Memória

Um bom DataCollator também otimiza o uso de memória. No código que você compartilhou, o `DynamicPaddingDataCollator` é um excelente exemplo disso:

- Em vez de preencher todas as sequências até um comprimento máximo fixo (por exemplo, 512 tokens)
- Ele calcula o comprimento máximo dentro do lote atual
- Preenche apenas até esse comprimento

Isso economiza muita memória quando a maioria das sequências em um lote é curta.

#### O Caso Específico do DynamicPaddingDataCollator

O `DynamicPaddingDataCollator` no seu código é uma extensão personalizada do `DataCollatorForLanguageModeling` padrão da Hugging Face. Ele:

1. Encontra o comprimento máximo dentro do lote atual
2. Para cada exemplo:
   - Calcula quanto preenchimento é necessário
   - Adiciona tokens de padding aos IDs de entrada
   - Adiciona zeros à máscara de atenção correspondente
   - Ou trunca se necessário (se a sequência for mais longa que o permitido)
3. Aplica a lógica de mascaramento do MLM
4. Garante que as formas e tipos dos tensores estão corretos usando a função `fix_batch_inputs`

#### Analogia para Entender Melhor

Pense no DataCollator como o gerente de uma mesa em um restaurante:

1. Várias pessoas (exemplos) chegam em grupos diferentes
2. O gerente precisa organizá-las em mesas (lotes) de tamanho fixo
3. Algumas mesas podem ter espaços vazios (padding)
4. Para um jogo especial durante o jantar (MLM), o gerente venda os olhos de algumas pessoas aleatoriamente

#### Por que o Dynamic Padding é Importante?

Considere dois cenários com 100 sequências, onde a maioria tem 20 tokens, mas algumas poucas têm 500:

1. **Padding Fixo**: Todas as 100 sequências são preenchidas até 500 tokens = 50.000 tokens totais
2. **Padding Dinâmico**: 
   - Lote 1: 16 sequências, max_len=20 → 320 tokens
   - Lote 2: 16 sequências, max_len=25 → 400 tokens
   - ...
   - Último lote: 4 sequências, max_len=500 → 2.000 tokens
   - Total muito menor que 50.000!

Essa economia permite:
- Treinar com lotes maiores
- Usar menos memória GPU
- Treinar modelos maiores
- Acelerar o treinamento

#### Conclusão

O DataCollator é uma peça crucial que prepara os dados brutos para o consumo eficiente pelo modelo. No caso do MLM, ele também implementa a estratégia de mascaramento que é central para o aprendizado. A versão com padding dinâmico que você está usando representa uma otimização inteligente que pode melhorar significativamente a eficiência do treinamento.

In [12]:
# --- Helper Function to Fix Batch Inputs ---
def fix_batch_inputs(inputs: dict) -> dict:
    """
    Esta função tem como objetivo garantir que os tensores de entrada tenham a forma e o tipo corretos:

    - Ela verifica três chaves importantes: "input_ids", "attention_mask" e "token_type_ids"
    - Remove dimensões extras (por exemplo, converte [1, batch, seq_len] para [batch, seq_len])
    - Converte "input_ids" para o tipo torch.long, que é o tipo esperado para IDs de tokens
    - Isso é importante porque inconsistências na forma dos tensores podem causar erros durante o treinamento
    """
    for key in ["input_ids", "attention_mask", "token_type_ids"]:
        if key in inputs:
            if inputs[key].dim() == 3 and inputs[key].shape[0] == 1:
                inputs[key] = inputs[key].squeeze(0)
            elif inputs[key].dim() > 2:
                raise ValueError(
                    f"Unexpected tensor shape for {key}: {inputs[key].shape}"
                )
    if "input_ids" in inputs and inputs["input_ids"].dtype != torch.long:
        inputs["input_ids"] = inputs["input_ids"].long()
    return inputs


# --- Forward Pass Function ---
def forward_pass(model, inputs):
    """
    Esta função realiza uma passagem para frente (forward pass) no modelo:

    - Primeiro, aplica a função fix_batch_inputs para garantir que as entradas estão corretas
    - Move os tensores para o dispositivo apropriado (CPU ou GPU)
    - Usa torch.amp.autocast para habilitar precisão mista automática quando estiver usando GPU
      - A precisão mista acelera o treinamento e reduz o uso de memória
    - Executa o modelo com as entradas e solicita que retorne um dicionário completo
    - Verifica se o modelo retornou uma perda (loss) e a retorna
    """
    inputs = fix_batch_inputs(inputs)
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.amp.autocast("cuda", enabled=(DEVICE.type == "cuda")):
        outputs = model(**inputs, return_dict=True)
    if outputs.loss is None:
        raise ValueError("Model did not return a loss.")
    return outputs.loss


# --- Evaluation Function ---
def evaluate(model, eval_dataset, data_collator, batch_size):
    """
    Esta função avalia o desempenho do modelo no conjunto de dados de avaliação:

    - Coloca o modelo em modo de avaliação (model.eval())
    - Itera sobre o conjunto de dados de avaliação em lotes
    - Para cada lote:
      - Desativa o cálculo de gradientes com torch.no_grad()
      - Usa precisão mista se estiver em GPU
      - Calcula a perda e a adiciona à lista de perdas
      - Captura e imprime erros, mas continua a avaliação
    - Retorna ao modo de treinamento (model.train())
    - Calcula e retorna a perda média
    """
    model.eval()
    losses = []
    eval_iterator = eval_dataset.iter(batch_size=batch_size)
    for batch in tqdm(eval_iterator, desc="Evaluating"):
        with torch.no_grad(), torch.amp.autocast(
            "cuda", enabled=(DEVICE.type == "cuda")
        ):
            inputs = data_collator(batch)
            try:
                loss = forward_pass(model, inputs)
                losses.append(loss.item())
            except Exception as e:
                print(f"Evaluation batch failed: {e}. Skipping.")
                continue
    model.train()
    average_loss = sum(losses) / len(losses) if losses else float("inf")
    return average_loss


class DynamicPaddingDataCollator(DataCollatorForLanguageModeling):
    """
    Esta classe estende DataCollatorForLanguageModeling e implementa um colator de dados com preenchimento dinâmico:

    - O preenchimento dinâmico significa que cada lote é preenchido apenas até o comprimento da sequência mais longa naquele lote específico
    - Isso é mais eficiente que usar um comprimento fixo para todos os lotes
    - Para cada exemplo no lote:
      - Calcula quanto padding é necessário
      - Adiciona tokens de padding aos IDs de entrada e zeros às máscaras de atenção
      - Ou trunca se necessário
    - Aplica a lógica de colação de dados do MLM (mascaramento aleatório de tokens)
    - Garante formas e tipos corretos com fix_batch_inputs
    """

    def __call__(self, examples: Dict[str, Any]) -> Dict[str, torch.Tensor]:
        # Find the maximum length within the current batch
        max_length = max(len(input_ids) for input_ids in examples["input_ids"])

        # Pad or truncate each example to the max_length
        batch = []
        input_ids = examples["input_ids"]
        attention_mask = examples["attention_mask"]

        for ids, mask in zip(input_ids, attention_mask):
            padding_length = max_length - len(ids)
            if padding_length > 0:
                # Pad
                ids = torch.tensor(ids + [self.tokenizer.pad_token_id] * padding_length)
                mask = torch.tensor(mask + [0] * padding_length)
            elif padding_length <= 0:
                # Truncate (if enabled in your tokenizer)
                ids = torch.tensor(ids[:max_length])
                mask = torch.tensor(mask[:max_length])

            batch.append({"input_ids": ids, "attention_mask": mask})

        # Apply the rest of the data collation logic (MLM masking, etc.)
        batch = self.torch_call(
            batch
        )  # Use torch_call instead of __call__ to call the parent's method

        # Ensure correct shapes and dtypes
        batch = fix_batch_inputs(batch)

        return batch

### Entendendo Chunks, Batches e Lotes no Treinamento de Modelos

#### Primeiro, vamos definir cada termo:

##### Lote (Batch)
"Lote" e "batch" são na verdade a mesma coisa - batch é simplesmente o termo em inglês para lote. No contexto de aprendizado de máquina:

- Um lote/batch é um grupo de exemplos que são processados juntos pelo modelo
- O tamanho do lote (batch size) é um hiperparâmetro importante que afeta a velocidade e a estabilidade do treinamento
- No código, você vê referências como `batch_size=training_config.per_device_train_batch_size`

##### Chunk
Um chunk é uma divisão maior do dataset, que contém múltiplos lotes. No seu código:

- O dataset inteiro é dividido em chunks
- Cada chunk é associado a uma probabilidade diferente de MLM
- Você vê isso em `chunk_size_dataset = available_size // num_chunks`

#### Como eles se relacionam (do maior para o menor):

1. **Dataset completo**: Todos os seus dados de treinamento
2. **Chunks**: Grandes divisões do dataset, cada um usado com uma configuração específica (ex: diferente probabilidade de MLM)  
3. **Lotes/Batches**: Pequenos grupos de exemplos processados juntos durante o treinamento


#### Por que essa estrutura é usada?

1. **Razão para usar chunks com diferentes mlm_probabilities**:
   - Diferentes taxas de mascaramento ajudam o modelo a aprender de forma mais robusta
   - Uma estratégia de "currículo" onde o modelo é exposto a desafios de dificuldade crescente

2. **Razão para usar lotes/batches**:
   - Limitações de memória: não é possível processar todos os dados de uma vez
   - Estabilidade de treinamento: atualizar os pesos após cada lote é mais estável que após cada exemplo
   - Eficiência computacional: processamento em paralelo de múltiplos exemplos

#### Fluxo Completo no Código:

1. O dataset é tokenizado (`tokenized_dataset`)
2. É dividido em `num_chunks` chunks
3. Para cada chunk:
   - É criado um data collator com uma probabilidade MLM específica
   - O chunk é dividido em lotes (batches)
   - Cada lote é processado, a perda é calculada, e os pesos do modelo são atualizados

Este método de "treinamento em camadas" (épocas → chunks → lotes) permite um treinamento mais eficiente e eficaz do modelo de linguagem mascarada.


In [13]:
print(f"Tamanho do dataset = {training_config.dataset_size}")
print(
    f"Serão definidos {training_config.num_chunks} chunks de {training_config.chunk_size} dados"
)
print(
    f"Cada chunk terá {training_config.chunk_train_size} dados de treino e {training_config.eval_size_per_chunk} dados de validação."
)
print(f"O número de dados de treino será, portanto: {training_config.available_size}")
print(f"O número de dados de validação será: {training_config.eval_size}")

Tamanho do dataset = 3456
Serão definidos 5 chunks de 691 dados
Cada chunk terá 622 dados de treino e 69 dados de validação.
O número de dados de treino será, portanto: 3111
O número de dados de validação será: 345


# Treinamento

In [14]:
def start_log(epoch, chunk_number, training_config):
    print(
        f"\nEpoch {epoch + 1}/{training_config.num_train_epochs} | "
        f"MLM Probability: {MLM_PROBABILITIES[chunk_number]}"
    )


def chunk_train_test_split(chunk_number, training_config):

    eval_start_idx = chunk_number * training_config.chunk_size
    eval_end_idx = eval_start_idx + training_config.eval_size_per_chunk - 1
    train_start_idx = (
        chunk_number * training_config.chunk_size + training_config.eval_size_per_chunk
    )
    train_end_idx = train_start_idx + training_config.chunk_train_size - 1

    print(
        f"\tSplitting | "
        f"chunk: {eval_start_idx}-{train_end_idx} | "
        f"eval: {eval_start_idx}-{eval_end_idx} | "
        f"train: {train_start_idx}-{train_end_idx}"
    )

    train_dataset = (
        tokenized_dataset.skip(train_start_idx)
        .take(training_config.chunk_train_size)
        .shuffle(seed=42)
    )

    eval_dataset = (
        tokenized_dataset.skip(eval_start_idx)
        .take(training_config.eval_size_per_chunk)
        .shuffle(seed=42)
    )

    return train_dataset, eval_dataset


def eval_loss(model, batch):
    inputs = data_collator(batch)
    loss = forward_pass(model, inputs)
    return loss


def update_model_parameters(scaler, optimizer, scheduler):
    """Atualiza os parâmetros do modelo, ajusta o learning rate e limpa os gradientes."""
    global global_step
    scaler.step(optimizer)
    scaler.update()
    scheduler.step()
    optimizer.zero_grad()
    torch.cuda.empty_cache()  # Limpa a memória da GPU
    global_step += 1


def evaluate_step(model, eval_dataset, data_collator, training_config):
    global global_step
    eval_interval = training_config.total_steps_per_epoch // (
        training_config.num_train_epochs * 4
    )
    if eval_interval > 0 and (global_step % eval_interval == 0):
        eval_loss = evaluate(
            model,
            eval_dataset,
            data_collator,
            batch_size=training_config.train_batch_size_per_device,
        )
        print(f"Evaluation loss at step {global_step}: {eval_loss}")


def push_to_hub(tokenizer, model):
    global global_step
    # Push to hub incl TESTING
    if global_step % PUSH_INTERVAL == 0:
        print(f"Saving and pushing model at step {global_step}...")
        model.save_pretrained(TRAINED_MODEL_PATH)
        tokenizer.save_pretrained(TRAINED_MODEL_PATH)
        print(f"Model saved and pushed at step {global_step}.")

In [15]:
LEARNING_RATE = 5e-3
WEIGHT_DECAY = 0.01
NUM_WARMUP_STEPS = 0

In [16]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)


# O torch.amp.GradScaler é uma ferramenta do PyTorch projetada para facilitar o treinamento de
# modelos utilizando precisão mista (mixed precision training). Em resumo, ele serve para
# escalonar (amplificar) os gradientes durante o backpropagation de forma a evitar problemas
# numéricos, como o underflow, que podem ocorrer ao usar representações de 16 bits (FP16).
# - Como Funciona
#   1. Escalonamento dos Gradientes: Ao multiplicar o valor do loss por um fator de escala,
#      os gradientes calculados ficam em uma faixa numérica mais segura, evitando que valores
#      muito pequenos se percam durante os cálculos.
#   2. Desescalonamento: Antes de atualizar os parâmetros do modelo, o GradScaler desfaz essa
#      multiplicação, garantindo que as atualizações ocorram com os valores corretos.
#   3. Ajuste Dinâmico da Escala: O GradScaler monitora a ocorrência de overflows (quando os
#      valores ficam excessivamente grandes) e ajusta automaticamente o fator de escala,
#      aumentando ou diminuindo conforme necessário para manter a estabilidade do treinamento.
scaler = torch.amp.GradScaler(DEVICE, enabled=(DEVICE.type == "cuda"))

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=NUM_WARMUP_STEPS,
    num_training_steps=training_config.total_train_steps,
)

In [ ]:
model.train()
global_step = 0

for epoch in range(training_config.num_train_epochs):
    for chunk_number, mlm_probability in enumerate(MLM_PROBABILITIES):
        start_log(epoch, chunk_number, training_config)
        data_collator = DynamicPaddingDataCollator(
            tokenizer, mlm_probability=mlm_probability
        )
        train_dataset, eval_dataset = chunk_train_test_split(
            chunk_number, training_config
        )
        train_iterator = train_dataset.iter(
            batch_size=training_config.train_batch_size_per_device
        )
        for step, batch in tqdm(
            enumerate(train_iterator), desc=f"Training (MLM {mlm_probability})"
        ):
            accumulation_step_complete = (
                step + 1
            ) % training_config.gradient_accumulation_steps == 0
            try:
                loss = eval_loss(model, batch)
                scaler.scale(
                    loss / training_config.gradient_accumulation_steps
                ).backward()
                if accumulation_step_complete:
                    update_model_parameters(scaler, optimizer, scheduler)
                    evaluate_step(model, eval_dataset, data_collator, training_config)
                    push_to_hub(tokenizer, model)

            except Exception as e:
                print(f"Training batch failed: {e}. Skipping.")
                continue


Epoch 1/3 | MLM Probability: 0.05
	Splitting | chunk: 0-690 | eval: 0-68 | train: 69-690


Training (MLM 0.05): 0it [00:00, ?it/s]

Evaluating: 0it [00:00, ?it/s]

Evaluation loss at step 36: nan


Evaluating: 0it [00:00, ?it/s]

...

Evaluation loss at step 612: 7.815092325210571

Epoch 2/3 | MLM Probability: 0.2
	Splitting | chunk: 2073-2763 | eval: 2073-2141 | train: 2142-2763


Training (MLM 0.2): 0it [00:00, ?it/s]

Evaluating: 0it [00:00, ?it/s]

Evaluation loss at step 648: 8.085047986772326


Evaluating: 0it [00:00, ?it/s]